<a href="https://colab.research.google.com/github/mhriyad99/Clinically-Adaptable-Machine-Learning-Model-To-Identify-Early-Appreciable-Feature-of-Diabetes/blob/main/Diabetes_Data_Analysis_1_(Collected_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')
plt.rc('axes', titlesize = 18)        # font size of title
plt.rc('axes', labelsize = 14)        # font size of x and y label
plt.rc('xtick', labelsize = 13)       # font size of tick label
plt.rc('ytick', labelsize = 13)       # font size of tick label
plt.rc('legend', fontsize = 13)       # legend fontsize
plt.rc('font', size = 13)             # controls default text size

In [ ]:
data = pd.read_csv("/content/Questionaries' for pre-diagnose diabetes (Responses) - Form Responses 1.csv")
data.head()

##**Data Cleaning**

In [ ]:
data.drop(['Timestamp', 'Height'], axis = 1, inplace = True)

In [ ]:
data.head()

In [ ]:
data.info()

Here we can see that 'Age' and 'Weight' columns are of object type. So, we have to convert them to numeric type. More specifically to float64

In [ ]:
data.isnull().sum()

From the above we can see there are 5 missing values in the 'Smoking' column.

In [ ]:
data['Smoking'].value_counts()

Because most of the values of the column 'Smoking' is 'No' so we will try to replace the 5 missing values with 'No' 

###Replacing Missing Values

In [ ]:
smoking = data.Smoking

In [ ]:
data.shape

In [ ]:
for i in range(data.shape[0]):
  if isinstance(data.loc[i, 'Smoking'], float):
    print(i)

In [ ]:
data.loc[[266,364,366,494,505],['Smoking']]

In [ ]:
for i in range(data.shape[0]):
  if isinstance(data.loc[i, 'Smoking'], float):
    data.loc[i, 'Smoking'] = 'No'

In [ ]:
data.isnull().sum()

In [ ]:
data['Age'][1:] = data['Age'][1:].astype(np.float64)

In [ ]:
data['Age'][1:].mean()

In [ ]:
data['Age'][0] = data['Age'][1:].mean() 

####**Cleaning coulumn 'Weight'**

In [ ]:
import re

for i in range(data.shape[0]):
  match = re.findall(r'\d*',data['Weight'][i])
  if match[0] == '':
    data['Weight'][i] = str(0)
  else:
    data['Weight'][i] = match[0]

In [ ]:
data.head()

In [ ]:
# Converting 'Age' and 'Weight' to numpy float
data['Age'] = data['Age'].astype(np.float64)
data['Weight'] = data['Weight'].astype(np.float64)

In [ ]:
data.dtypes

In [ ]:
data['Weight'].mean()
   

In [ ]:
for i in range(data.shape[0]):
  if data['Weight'][i] == 0:
    print(i)
    data['Weight'][i] == data['Weight'].mean()
 

##**Exploratory Data Analysis**

In [ ]:
columns = data.columns

In [ ]:
for i in columns:
  if data[i].dtypes == 'O':
    X = data[i].value_counts()
    plt.figure()
    plt.bar(X.index,X.values, width= 0.4)
    for j in range (len(X.values)):
      plt.text(j,X.values[j], s= X.values[j], ha = 'center', va = 'bottom')
    plt.title(i)
  else:
    fig, ax = plt.subplots(1, 1)
    ax.hist(data[i], bins = [0,10,20,30,40,50,60,70,80,90,100])
  
    # Set title
    ax.set_title(i)
  
    # Make some labels.
    rects = ax.patches
    for rect in rects:
      height = rect.get_height()
      ax.text(rect.get_x() + rect.get_width() / 2, height+0.01, height,
              ha='center', va='bottom')


##**Encoding Categorical features**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in columns:
  if data[column].dtypes == 'O':
    data[column] = le.fit_transform(data[column])

In [ ]:
data.head()

##**Correlation Analysis**

In [ ]:
plt.figure()
plt.figure(figsize = (25,10))
sns.heatmap(data.corr(), cmap = 'inferno', annot = True)

From above we can see that coulumn :


*  **Age** is highly correlated with coulumn **High Blood Pressure**, **Dalayed Healing** and it is also a significant feature in detecting **Diabetes**.
*   **High Blood Pressure** have good correlation coefficient of **0.61** with **Diabetes**.
*   **Polyuria** have good correalation coefficient of **0.62** with **Delayed Healing** and it is also a significant feature for detecting **Diabetes**.
*   **Delayed Healing** is highly correalated with **Diabetes**, with correlation coefficient of **0.76**.







##**t-Test**


In [ ]:
cols = list(data.columns)
from scipy import stats

for i in columns:
  for j in columns:
    if cols.index(j) <= cols.index(i):
      pass
    else:
      t_test, p_value = stats.ttest_ind(a = data[i], b = data[j], equal_var= False)
      if p_value > 0.05:  
        print("\n")
        print(">>> t-test between {} and {}:".format(i,j))
        print("The t-test value = ",t_test, " and p-value= ", p_value)

##**Outlier Detection**

In [ ]:
def univariate_outlier_detection(column):

  print("\n\n-----------------------  {n}  ------------------------".format(n = column))

  mean = data[column].mean()
  median = data[column].median()
  mode = data[column].mode()

  print("\nMean:", mean, "\nMedian: ", median,"\nMode: ", mode[0])

  Q1 = data[column].quantile(q = 0.25)
  Q3 = data[column].quantile(q = 0.75)
  IQR = stats.iqr(data[column])

  print("\n\n1st quartile: ", Q1)
  print("3rd quartile: ", Q3)
  print("Inter-quartile Range: ", IQR)

  l_outlier_range = Q1 - 1.5*IQR
  u_outlier_range = Q3 + 1.5*IQR
  print("\n\nLower outlier range: ", l_outlier_range)
  print("Upper outlier range: ", u_outlier_range)

  print("\n\nNumber of outlier in upper range: ", data[data[column] > u_outlier_range][column].count())
  print("Number of outlier in lower range: ", data[data[column] < l_outlier_range][column].count())
  print("\n% of outlier in upper range: ", (data[data[column] > u_outlier_range][column].count()*100/len(data)))
  print("% of outlier in lower range: ", (data[data[column] < l_outlier_range][column].count()*100/len(data)))

  #Boxplot
  print("Boxplot of column " + column)
  plt.figure()
  ax = sns.boxplot(x = data[column])

  plt.show()

In [ ]:
for i in columns:
  if data[i].dtypes == 'float64':
    univariate_outlier_detection(i)

###Using Classifiers

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score